#### Architecture - Semantic Search using Elastic Search


![image.png](attachment:image.png)

* Two very important concepts in Elasticsearch are documents and indexes.

* A document is collection of fields with their associated values.

* To work with Elasticsearch you have to organize your data into documents, and then add all your documents to an index.

* Index as a collection of documents that is stored in a highly optimized format designed to perform efficient searches.

#### Step 1: Prepare documents

In [14]:
%pip install -r requirements.txt

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.2/480.2 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 111.9 MB/

In [1]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [2]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

#### Step 2: Create Embeddings using Pretrained Models

Sentence Transformers documentation here: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

In [3]:
%pip uninstall numpy torch sentence_transformers -y

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: torch 2.3.1+cu121
Uninstalling torch-2.3.1+cu121:
  Successfully uninstalled torch-2.3.1+cu121


In [4]:
%pip install numpy==1.26.4 --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 150.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 0.32.1 requires torch>=1.10.0, which is not installed.
torchtext 0.18.0 requires torch>=2.3.0, which is not installed.
torchvision 0.18.1+cu121 requires torch==2.3.1, which is not installed.
xgboost 2.1.0 requires nvidia-nccl-cu12; platform_system == "Linux" and platform_machine != "aarch64", which is not installed.


In [5]:
%pip install torch --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 259.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 187.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 188.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 127.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 301.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 152.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 222.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 228.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 278.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 288.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 323.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [6]:
%pip install sentence_transformers==2.7.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.0 MB/s eta 0:00:00


In [7]:
# This is a new library compared to the previous modules.
# Please perform "pip install sentence_transformers==2.7.0"
from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
len(model.encode("This is a simple sentence"))

768

In [9]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

#### Step 3: Setup ElasticSearch connection

In [15]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

es_client.info()

ObjectApiResponse({'name': '1d6a54f3bf1e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Ipw05X0MTjmreaKWB_PNiA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

#### Step 4: Create Mappings and Index

* Mapping is the process of defining how a document, and the fields it contains, are stored and indexed.

* Each document is a collection of fields, which each have their own data type.

* We can compare mapping to a database schema in how it describes the fields and properties that documents hold, the datatype of each field (e.g., string, integer, or date), and how those fields should be indexed and stored



In [16]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [17]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

#### Step 5: Add documents into index

In [18]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

#### Step 6: Create end user query

In [19]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [26]:
# GET course-questions/_search
# {
#   "knn": {
#       "field": "text_vector",
#       "query_vector":[-6.35388270e-02,  2.65322383e-02,  9.98951774e-03, -1.36715230e-02,
#        -1.28370123e-02,  9.20394342e-03, -8.48790340e-04, -3.05588525e-02,
#        -2.46247253e-03,  1.72226671e-02,  4.31264490e-02,  8.57972633e-03,
#        -1.93418711e-02,  1.04178712e-02, -4.21265438e-02,  1.33134099e-02,
#         9.56985131e-02, -3.55436206e-02,  1.66664384e-02, -1.86405051e-02,
#        -5.06158033e-03,  1.73184033e-02, -1.59354433e-02, -2.41835881e-02,
#        -1.45203825e-02, -3.99563834e-02, -7.25806206e-02,  2.68063098e-02,
#        -5.12384288e-02,  5.11277616e-02, -3.41739468e-02,  1.96610428e-02,
#         2.75925975e-02,  4.62351777e-02,  1.54686859e-06, -1.40663497e-02,
#         1.84368975e-02,  4.19799145e-03,  5.42885903e-03,  1.89430341e-02,
#         3.05294432e-02,  4.25610691e-02,  1.21712417e-03, -8.74176528e-03,
#         1.01104733e-02, -2.74074171e-02, -2.32483540e-02,  3.35225873e-02,
#        -2.28760634e-02,  2.62946337e-02, -2.22461335e-02,  5.40544139e-03,
#        -1.50831845e-02, -9.47785936e-03,  7.65770301e-03,  1.75578194e-03,
#         2.21968554e-02, -8.82813241e-03,  7.30441213e-02, -4.51871790e-02,
#         2.32994347e-03, -4.73779030e-02, -2.81618033e-02, -6.09941483e-02,
#         1.16178710e-02,  6.38647825e-02,  6.53979108e-02,  1.50072039e-03,
#        -3.37890536e-02, -4.22633812e-02, -2.23228466e-02,  9.00195837e-02,
#         2.28384207e-03,  7.91870151e-03, -9.18731373e-03,  3.82849202e-02,
#         4.30368558e-02,  3.61037515e-02, -2.15227460e-03, -1.52776372e-02,
#        -4.29502279e-02, -1.42295947e-02,  1.59920212e-02,  3.97709534e-02,
#        -2.23183185e-02, -4.24561948e-02, -9.47693177e-03, -1.07099826e-03,
#         2.58649494e-02,  2.12642681e-02,  4.30124737e-02, -4.21220101e-02,
#        -5.07041393e-03,  1.82438456e-02, -8.16201139e-03, -1.48695307e-02,
#        -1.70592833e-02,  7.58345872e-02, -8.20751563e-02, -4.50881943e-02,
#        -8.71965289e-03, -4.49553393e-02,  3.77592519e-02,  2.09175684e-02,
#        -1.09735681e-02,  6.73758565e-03,  3.60601768e-02,  1.08690271e-02,
#         1.09228734e-02,  3.87176648e-02,  4.92368964e-03, -1.57710090e-02,
#        -6.04422800e-02,  5.04793897e-02, -9.61321034e-03, -2.62233522e-02,
#        -2.08928194e-02, -2.64143851e-02,  3.68586485e-03, -1.02991369e-02,
#         6.78830966e-02, -2.05396619e-02,  1.88170504e-02,  3.83320637e-02,
#        -4.66197357e-02,  2.47132648e-02, -9.67195164e-03,  4.44084965e-02,
#         3.57156135e-02,  3.82395694e-03,  5.84052801e-02,  3.98035794e-02,
#        -3.18336934e-02, -5.25367446e-02,  1.53399669e-02,  1.81686673e-02,
#        -3.05904076e-02,  6.54080883e-02, -6.09459803e-02, -3.12423185e-02,
#        -2.64271013e-02, -9.39445756e-03, -2.47351080e-02, -1.15903770e-03,
#        -2.64298487e-02,  3.69182788e-02,  1.67394578e-02,  5.80479577e-02,
#        -5.72493009e-04,  4.65484075e-02,  1.60511062e-02,  8.88596755e-03,
#         1.51253054e-02,  8.64263251e-03,  3.97731327e-02,  6.67994536e-05,
#        -4.56167385e-02,  5.46206273e-02, -2.35528462e-02,  9.65549201e-02,
#         7.02239340e-03,  4.09134142e-02,  9.57184844e-03, -2.31743753e-02,
#        -5.06861322e-02, -4.51900810e-02,  2.09901594e-02,  4.20809872e-02,
#         8.88705812e-03, -7.13691190e-02, -5.16369054e-03,  6.17568120e-02,
#         1.72940586e-02, -1.36941462e-03,  1.39626870e-02, -1.05941081e-02,
#        -2.38400102e-02, -5.85919283e-02, -8.00424162e-03, -2.11199317e-02,
#         7.48951286e-02, -1.06796987e-01, -1.20697068e-02,  2.22305115e-02,
#         1.27509981e-02,  9.43246391e-03, -5.63748530e-04, -3.37913893e-02,
#         3.74825187e-02, -2.54555792e-03, -4.97520156e-02, -4.37055081e-02,
#         2.50301138e-02,  8.25663656e-03,  9.25713684e-03, -2.44841427e-02,
#        -1.37027884e-02, -3.44014913e-02,  9.97631252e-03,  1.71746239e-02,
#        -9.37420949e-02,  1.52915111e-02,  1.31554510e-02,  2.21383292e-02,
#        -4.17943150e-02, -2.77680159e-03,  3.65033075e-02, -4.06630598e-02,
#         1.85582004e-02,  1.25366189e-02,  4.92950641e-02,  1.61770396e-02,
#         5.77118658e-02,  1.02912532e-02,  3.15833315e-02, -1.22766541e-02,
#        -1.91974491e-02, -6.26271823e-03,  3.83149981e-02,  2.54966430e-02,
#        -1.84738319e-02, -3.67629677e-02,  7.77552184e-03,  5.22513408e-03,
#         4.59131412e-02,  2.25596856e-02, -1.15516037e-02, -1.90421287e-03,
#        -6.96150512e-02, -4.04684246e-02, -4.22044471e-02,  1.06172115e-02,
#         4.71916981e-02, -1.18402420e-02, -1.58141572e-02,  1.70259678e-04,
#         6.93977950e-03, -2.04482209e-02,  2.44037230e-02, -6.35038223e-03,
#         2.45455513e-03,  6.10066503e-02,  5.24352817e-03, -5.93643682e-03,
#        -4.85642850e-02, -6.43062592e-02, -2.77552456e-02, -2.32213419e-02,
#        -5.15835769e-02, -7.99825117e-02, -7.21247569e-02, -8.55054799e-03,
#         4.75817025e-02,  9.95598733e-03, -7.41919316e-03,  1.54344290e-02,
#        -1.66547559e-02, -1.25077355e-03, -5.43871336e-02, -5.77903502e-02,
#         8.82528573e-02, -2.33616326e-02, -5.00978492e-02, -1.02615533e-02,
#         7.30055291e-03,  3.43292281e-02, -2.12623011e-02, -8.88561010e-02,
#         6.11365447e-03, -1.52316839e-02, -2.80353501e-02, -3.95468250e-02,
#        -3.22616100e-02, -1.33498395e-02,  3.01755173e-03, -1.85572114e-02,
#        -3.35369371e-02, -4.75610271e-02,  2.37506256e-02, -9.55078052e-04,
#        -2.07520872e-02,  6.79148063e-02, -1.58417374e-02, -4.85723931e-03,
#         5.24530327e-03, -4.89978446e-03,  9.10718646e-03,  1.87337515e-03,
#        -2.99504660e-02,  3.51611711e-03, -6.60958067e-02,  6.41788030e-03,
#         2.22590715e-02,  7.53074437e-02,  3.53788659e-02, -5.28311096e-02,
#         4.67569157e-02, -8.92656446e-02, -3.08805760e-02,  7.91029818e-03,
#         5.52251898e-02,  8.22758600e-02,  2.11164225e-02,  5.53627200e-02,
#         6.49390668e-02, -4.04138789e-02, -2.28957622e-04, -7.43037388e-02,
#         1.81916654e-02,  5.70268296e-02,  1.26477024e-02, -4.36689192e-03,
#         3.70855466e-03,  1.08027523e-02, -6.38372228e-02,  2.36931257e-02,
#        -3.11889723e-02, -4.69807237e-02,  1.30222142e-02,  4.47530393e-03,
#        -3.15080285e-02, -5.37434071e-02,  3.28278802e-02, -9.71143600e-03,
#        -3.07110866e-04,  8.38305950e-02, -1.95403229e-02,  5.09084342e-03,
#        -1.97637733e-02, -3.13787758e-02, -4.07412723e-02,  3.22756910e-04,
#        -8.61327630e-03, -3.75105515e-02, -4.59784232e-02,  6.17342107e-02,
#        -3.70969111e-03,  3.80500755e-03,  4.61920090e-02,  6.89788908e-02,
#        -6.14398606e-02,  4.04656213e-03,  1.93972848e-02, -8.83569419e-02,
#        -1.91511046e-02, -3.36884987e-03,  3.12168673e-02,  2.79408339e-02,
#         2.69571636e-02,  2.84631699e-02,  5.44577315e-02,  4.18368541e-03,
#        -4.36899811e-03, -5.60212275e-03,  1.00393472e-02,  2.35472689e-03,
#        -2.89809946e-02,  5.90746105e-03,  8.33062083e-03, -1.89391002e-02,
#        -3.17817368e-02,  1.79902986e-02,  1.18134692e-02,  6.12824596e-02,
#        -5.50248325e-02, -1.94575135e-02, -3.05044185e-03, -2.62853340e-03,
#        -4.01911959e-02, -3.97680327e-02,  5.51756844e-02,  3.65122147e-02,
#         9.05991625e-03,  3.22187170e-02,  3.10144504e-03, -2.33913977e-02,
#        -9.58415587e-03, -1.85423996e-03, -2.90373154e-02,  8.50840006e-03,
#         7.87928849e-02, -5.30835055e-02, -1.31328367e-02,  1.28030861e-02,
#         5.27247787e-02,  2.92754024e-02, -6.62970776e-03,  8.29944015e-02,
#         2.23353747e-02, -2.35364661e-02, -1.10355355e-02,  5.67874755e-04,
#         1.05503341e-02, -1.09110966e-01,  1.64827462e-02, -6.21259287e-02,
#         4.83694337e-02, -1.18169328e-02, -1.38406791e-02,  4.21534106e-02,
#         3.08368709e-02,  9.55001935e-02,  7.69733563e-02, -3.31638269e-02,
#         2.51529049e-02, -1.05492037e-03,  5.55963628e-02, -2.95078270e-02,
#        -8.78651626e-03, -3.61278467e-02,  3.98420058e-02,  2.47338936e-02,
#         4.00946736e-02,  2.51627900e-02, -1.98711511e-02,  5.13324104e-02,
#         1.96090732e-02, -4.06155810e-02, -4.59468178e-02,  6.02422394e-02,
#         1.91238876e-02,  1.41611304e-02,  1.49158379e-02,  8.63641035e-03,
#         1.24415727e-02, -2.85918228e-02, -3.90159003e-02, -2.25124732e-02,
#         3.73598821e-02,  3.57946986e-03, -1.60144060e-03, -9.97179300e-02,
#         1.41539723e-02, -5.37276566e-02,  9.28293914e-03, -5.70615567e-02,
#         1.60645917e-02, -1.79864559e-02, -3.83544788e-02, -2.85584480e-02,
#        -5.52239381e-02, -5.26455343e-02, -3.01631670e-02, -5.46472482e-02,
#        -1.45087689e-02, -8.18924140e-03,  3.82901356e-02, -3.51964347e-02,
#        -6.40672818e-02,  2.63199247e-02,  4.67148833e-02, -7.60204941e-02,
#         5.67530505e-02, -5.63785844e-02,  3.04889586e-02,  3.73169756e-03,
#         4.29984480e-02,  3.09336018e-02, -2.14926545e-02,  1.33711249e-02,
#         7.02268723e-03, -7.32180139e-04, -2.93228179e-02,  1.82911903e-02,
#         4.95046079e-02, -5.13419099e-02,  1.36850690e-02, -7.72360945e-03,
#         2.50036120e-02,  4.18160670e-02, -1.66418552e-02, -3.51010412e-02,
#         2.63861045e-02,  1.55906389e-02, -7.28965830e-03,  3.54819298e-02,
#        -4.35528830e-02,  4.94477078e-02,  2.14855727e-02, -3.27146202e-02,
#        -2.45707296e-03, -2.30731834e-02, -2.86313239e-04, -1.85405146e-02,
#        -3.26399994e-03, -6.73678704e-03, -2.20459364e-02, -1.77117512e-02,
#         2.99205836e-02, -5.36527149e-02, -2.90003549e-02, -2.83161178e-03,
#         1.82511397e-02, -4.49408330e-02,  5.60665242e-02,  1.03998818e-02,
#        -5.98358661e-02, -1.39806625e-02, -6.36844784e-02,  6.67121187e-02,
#        -1.58022512e-02,  5.68064116e-02,  5.82683599e-03,  6.09616600e-02,
#         6.19807839e-02,  2.15773266e-02, -6.15327805e-02,  1.66593213e-02,
#        -4.41254452e-02, -1.13536883e-02, -1.14786252e-02, -2.17435788e-02,
#        -1.97188631e-02, -4.91952747e-02,  9.38986689e-02,  5.29194474e-02,
#         2.54839361e-02, -5.69909476e-02, -7.05301389e-02,  3.56271155e-02,
#         2.58147027e-02, -5.36590107e-02,  2.65945662e-02,  7.41247982e-02,
#         3.92559692e-02, -6.16428368e-02,  2.26533599e-02, -3.93039472e-02,
#         4.47604023e-02,  3.72001491e-02, -5.27060404e-02, -5.08112200e-02,
#        -1.77795291e-02, -1.86528638e-02, -2.48605385e-02,  9.22461525e-02,
#        -3.99115905e-02,  3.14114057e-02, -2.24148408e-02, -6.05608104e-03,
#         4.59699798e-03,  6.55871257e-03,  1.82527874e-03, -8.05642735e-03,
#        -3.05913528e-03,  2.47964859e-02,  4.71892580e-02,  2.85140295e-02,
#         2.41699368e-02,  1.72727033e-02,  7.47751305e-03,  9.29769874e-02,
#        -7.47835189e-02,  1.83949247e-02, -8.50671157e-02, -5.06101709e-33,
#        -1.56907756e-02,  3.60210203e-02,  4.21261322e-03, -3.92971449e-02,
#        -3.22588049e-02,  2.03947648e-02,  6.77641183e-02,  8.95519275e-04,
#        -5.86214624e-02,  1.75790749e-02, -1.09964032e-02,  2.29445770e-02,
#         2.18059029e-02,  1.98075734e-02,  3.00855525e-02, -5.35030849e-03,
#         3.29179540e-02,  2.51705628e-02,  8.52273125e-03, -2.76516359e-02,
#         2.47744787e-02,  6.13632219e-05, -7.34299496e-02, -1.27970884e-02,
#         2.65562199e-02,  1.98717657e-02,  3.23472321e-02, -5.73546439e-02,
#        -1.58187002e-02, -1.52053703e-02,  4.43447530e-02,  2.79282425e-02,
#        -2.18765475e-02, -1.45105820e-03,  2.73276994e-04,  6.27390156e-03,
#        -2.46591195e-02, -4.01210180e-03,  2.26378143e-02,  3.14928032e-02,
#        -4.38302616e-03, -1.02146538e-02,  2.70649381e-02,  1.26182707e-02,
#         1.60928294e-02,  1.01417759e-02,  3.12726311e-02,  1.78395249e-02,
#        -5.88438287e-02,  3.23857996e-04,  3.15098427e-02, -1.09399436e-02,
#         1.30759673e-02,  2.88387947e-02, -1.98077876e-02,  4.71308567e-02,
#        -3.41147296e-02,  4.84242709e-03,  5.28466247e-04,  2.72621531e-02,
#         3.21750902e-03, -2.33742297e-02,  3.30080763e-02,  3.91170895e-03,
#        -1.13639552e-02,  9.49568115e-03,  2.39290129e-02, -2.84808762e-02,
#         4.16205525e-02, -2.69255531e-03, -4.31079008e-02,  3.59808095e-02,
#         5.03739156e-02, -2.85110250e-02, -2.84249466e-02,  4.52170242e-03,
#        -5.73280454e-02,  3.14072967e-02,  1.40482569e-02, -3.99632491e-02,
#        -9.78091061e-02,  7.01613203e-02, -2.46890783e-02,  2.98228581e-02,
#        -2.07746532e-02,  7.85104465e-04,  1.26001996e-03, -4.57690470e-02,
#        -9.89804231e-03,  1.88390445e-02, -1.46243749e-02,  3.76008116e-02,
#         2.75905207e-02, -1.09142335e-02,  9.39014554e-03, -5.32136969e-02,
#         5.25716972e-03,  3.80616374e-02, -3.88914496e-02, -2.09589079e-02,
#        -1.60054192e-02,  3.36383767e-02,  5.13529703e-02,  1.75285895e-04,
#         7.49087601e-04, -1.24496426e-02, -6.30771369e-02, -1.63804498e-02,
#        -3.62837948e-02, -1.08360564e-02, -3.04921344e-03,  5.65082878e-02,
#         6.99948892e-03,  2.80238315e-02,  5.78210726e-02,  4.96611977e-03,
#         3.77025874e-03, -2.84294374e-02, -3.26208137e-02, -6.75238594e-02,
#        -1.84433982e-02, -1.36077791e-01, -5.42167723e-02,  1.08281178e-02,
#        -1.77369249e-04,  3.75885814e-02, -1.12027479e-02,  3.95341814e-02,
#        -5.56724258e-02, -4.61753719e-02, -1.60338376e-02,  1.13554979e-02,
#         2.08576196e-07,  5.81366569e-02, -5.43791316e-02,  6.54743910e-02,
#         5.24706915e-02, -3.55250537e-02, -4.66826148e-02,  7.08852801e-03,
#         3.26832421e-02,  2.69151907e-02,  2.68478505e-02, -4.30063158e-02,
#        -7.80400354e-03, -3.43841431e-03,  5.41962758e-02,  2.54194494e-02,
#         7.08701741e-03,  1.75959466e-03,  8.17671418e-02, -2.86811162e-02,
#        -1.35912988e-02,  2.73381174e-02,  2.69771479e-02, -8.71380791e-03,
#         2.70841047e-02,  2.04075351e-02, -3.18201259e-03, -1.30775534e-02,
#        -1.41084762e-02,  5.35203237e-03, -7.44392499e-02, -1.27584515e-02,
#         1.18300496e-02, -5.95305264e-02,  1.56116411e-02, -6.03072438e-03,
#        -2.05468331e-02,  2.64899042e-02,  2.72203349e-02, -3.82143655e-04,
#         2.83557773e-02,  1.31781921e-02,  9.87495854e-03, -2.43306924e-02,
#        -2.53265142e-03,  6.56142682e-02,  3.04429140e-02,  1.30092725e-02,
#         1.81903429e-02, -1.87106710e-03, -3.11279558e-02,  9.91685688e-03,
#         1.95655841e-02, -1.80514937e-03,  2.22536065e-02,  2.46514054e-03,
#         9.54501238e-03, -5.87863429e-03,  2.27531651e-03, -5.49062863e-02,
#         1.68025296e-03,  2.45665144e-02,  1.04704984e-02, -3.63097899e-02,
#        -3.58516425e-02,  4.03851904e-02, -8.18642671e-04, -3.32758874e-02,
#         1.14417994e-34, -6.96108639e-02,  1.84117947e-02, -1.90845784e-02,
#         5.90707809e-02, -6.85224216e-03,  2.64697298e-02,  4.39649541e-03,
#        -3.31841619e-03, -3.24860751e-03,  7.86198583e-03, -6.80046761e-03],
#       "k": 5,
#       "num_candidates": 10000
#     }
#   }

In [20]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000,
}

res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

In [22]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'H49mGpEB8zadGYYapAui',
  '_score': 0.71479183,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'Mo9pGpEB8zadGYYalg7I',
  '_score': 0.6134736,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares a

#### Step 7: Perform Keyword search with Semantic Search (Hybrid/Advanced Search)

In [29]:
response = es_client.search(
    index=index_name,
    query={
        "bool": {
            "must": {
                "multi_match": {
                    "query": "windows or python?",
                    "fields": ["text", "question", "course", "title"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp",
                }
            }
          },
    }
)

In [31]:
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'ZY9mGpEB8zadGYYa6gtX',
  '_score': 7.728908,
  '_source': {'text': 'Problem: If you have already installed pgcli but bash doesn\'t recognize pgcli\nOn Git bash: bash: pgcli: command not found\nOn Windows Terminal: pgcli: The term \'pgcli\' is not recognized…\nSolution: Try adding a Python path C:\\Users\\...\\AppData\\Roaming\\Python\\Python39\\Scripts to Windows PATH\nFor details:\nGet the location: pip list -v\nCopy C:\\Users\\...\\AppData\\Roaming\\Python\\Python39\\site-packages\n3. Replace site-packages with Scripts: C:\\Users\\...\\AppData\\Roaming\\Python\\Python39\\Scripts\nIt can also be that you have Python installed elsewhere.\nFor me it was under c:\\python310\\lib\\site-packages\nSo I had to add c:\\python310\\lib\\Scripts to PATH, as shown below.\nPut the above path in "Path" (or "PATH") in System Variables\nReference: https://stackoverflow.com/a/68233660',
   'section': 'Module 1: Docker and Terraform',
   'question': 'PGCLI - pg

In [23]:
# Note: I made a minor modification to the query shown in the notebook here
# (compare to the one shown in the video)
# Included "knn" in the search query (to perform a semantic search) along with the filter
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [28]:
# https://www.elastic.co/search-labs/tutorials/search-tutorial/vector-search/hybrid-search
# GET course-questions/_search
# {
#   "size": 5,
#   "query": {
#     "match": {
#       "section": "General course-related questions"
#     }
#   },
#   "knn": {
#       "field": "text_vector",
#       "query_vector":[-6.35388270e-02,  2.65322383e-02,  9.98951774e-03, -1.36715230e-02,
#        -1.28370123e-02,  9.20394342e-03, -8.48790340e-04, -3.05588525e-02,
#        -2.46247253e-03,  1.72226671e-02,  4.31264490e-02,  8.57972633e-03,
#        -1.93418711e-02,  1.04178712e-02, -4.21265438e-02,  1.33134099e-02,
#         9.56985131e-02, -3.55436206e-02,  1.66664384e-02, -1.86405051e-02,
#        -5.06158033e-03,  1.73184033e-02, -1.59354433e-02, -2.41835881e-02,
#        -1.45203825e-02, -3.99563834e-02, -7.25806206e-02,  2.68063098e-02,
#        -5.12384288e-02,  5.11277616e-02, -3.41739468e-02,  1.96610428e-02,
#         2.75925975e-02,  4.62351777e-02,  1.54686859e-06, -1.40663497e-02,
#         1.84368975e-02,  4.19799145e-03,  5.42885903e-03,  1.89430341e-02,
#         3.05294432e-02,  4.25610691e-02,  1.21712417e-03, -8.74176528e-03,
#         1.01104733e-02, -2.74074171e-02, -2.32483540e-02,  3.35225873e-02,
#        -2.28760634e-02,  2.62946337e-02, -2.22461335e-02,  5.40544139e-03,
#        -1.50831845e-02, -9.47785936e-03,  7.65770301e-03,  1.75578194e-03,
#         2.21968554e-02, -8.82813241e-03,  7.30441213e-02, -4.51871790e-02,
#         2.32994347e-03, -4.73779030e-02, -2.81618033e-02, -6.09941483e-02,
#         1.16178710e-02,  6.38647825e-02,  6.53979108e-02,  1.50072039e-03,
#        -3.37890536e-02, -4.22633812e-02, -2.23228466e-02,  9.00195837e-02,
#         2.28384207e-03,  7.91870151e-03, -9.18731373e-03,  3.82849202e-02,
#         4.30368558e-02,  3.61037515e-02, -2.15227460e-03, -1.52776372e-02,
#        -4.29502279e-02, -1.42295947e-02,  1.59920212e-02,  3.97709534e-02,
#        -2.23183185e-02, -4.24561948e-02, -9.47693177e-03, -1.07099826e-03,
#         2.58649494e-02,  2.12642681e-02,  4.30124737e-02, -4.21220101e-02,
#        -5.07041393e-03,  1.82438456e-02, -8.16201139e-03, -1.48695307e-02,
#        -1.70592833e-02,  7.58345872e-02, -8.20751563e-02, -4.50881943e-02,
#        -8.71965289e-03, -4.49553393e-02,  3.77592519e-02,  2.09175684e-02,
#        -1.09735681e-02,  6.73758565e-03,  3.60601768e-02,  1.08690271e-02,
#         1.09228734e-02,  3.87176648e-02,  4.92368964e-03, -1.57710090e-02,
#        -6.04422800e-02,  5.04793897e-02, -9.61321034e-03, -2.62233522e-02,
#        -2.08928194e-02, -2.64143851e-02,  3.68586485e-03, -1.02991369e-02,
#         6.78830966e-02, -2.05396619e-02,  1.88170504e-02,  3.83320637e-02,
#        -4.66197357e-02,  2.47132648e-02, -9.67195164e-03,  4.44084965e-02,
#         3.57156135e-02,  3.82395694e-03,  5.84052801e-02,  3.98035794e-02,
#        -3.18336934e-02, -5.25367446e-02,  1.53399669e-02,  1.81686673e-02,
#        -3.05904076e-02,  6.54080883e-02, -6.09459803e-02, -3.12423185e-02,
#        -2.64271013e-02, -9.39445756e-03, -2.47351080e-02, -1.15903770e-03,
#        -2.64298487e-02,  3.69182788e-02,  1.67394578e-02,  5.80479577e-02,
#        -5.72493009e-04,  4.65484075e-02,  1.60511062e-02,  8.88596755e-03,
#         1.51253054e-02,  8.64263251e-03,  3.97731327e-02,  6.67994536e-05,
#        -4.56167385e-02,  5.46206273e-02, -2.35528462e-02,  9.65549201e-02,
#         7.02239340e-03,  4.09134142e-02,  9.57184844e-03, -2.31743753e-02,
#        -5.06861322e-02, -4.51900810e-02,  2.09901594e-02,  4.20809872e-02,
#         8.88705812e-03, -7.13691190e-02, -5.16369054e-03,  6.17568120e-02,
#         1.72940586e-02, -1.36941462e-03,  1.39626870e-02, -1.05941081e-02,
#        -2.38400102e-02, -5.85919283e-02, -8.00424162e-03, -2.11199317e-02,
#         7.48951286e-02, -1.06796987e-01, -1.20697068e-02,  2.22305115e-02,
#         1.27509981e-02,  9.43246391e-03, -5.63748530e-04, -3.37913893e-02,
#         3.74825187e-02, -2.54555792e-03, -4.97520156e-02, -4.37055081e-02,
#         2.50301138e-02,  8.25663656e-03,  9.25713684e-03, -2.44841427e-02,
#        -1.37027884e-02, -3.44014913e-02,  9.97631252e-03,  1.71746239e-02,
#        -9.37420949e-02,  1.52915111e-02,  1.31554510e-02,  2.21383292e-02,
#        -4.17943150e-02, -2.77680159e-03,  3.65033075e-02, -4.06630598e-02,
#         1.85582004e-02,  1.25366189e-02,  4.92950641e-02,  1.61770396e-02,
#         5.77118658e-02,  1.02912532e-02,  3.15833315e-02, -1.22766541e-02,
#        -1.91974491e-02, -6.26271823e-03,  3.83149981e-02,  2.54966430e-02,
#        -1.84738319e-02, -3.67629677e-02,  7.77552184e-03,  5.22513408e-03,
#         4.59131412e-02,  2.25596856e-02, -1.15516037e-02, -1.90421287e-03,
#        -6.96150512e-02, -4.04684246e-02, -4.22044471e-02,  1.06172115e-02,
#         4.71916981e-02, -1.18402420e-02, -1.58141572e-02,  1.70259678e-04,
#         6.93977950e-03, -2.04482209e-02,  2.44037230e-02, -6.35038223e-03,
#         2.45455513e-03,  6.10066503e-02,  5.24352817e-03, -5.93643682e-03,
#        -4.85642850e-02, -6.43062592e-02, -2.77552456e-02, -2.32213419e-02,
#        -5.15835769e-02, -7.99825117e-02, -7.21247569e-02, -8.55054799e-03,
#         4.75817025e-02,  9.95598733e-03, -7.41919316e-03,  1.54344290e-02,
#        -1.66547559e-02, -1.25077355e-03, -5.43871336e-02, -5.77903502e-02,
#         8.82528573e-02, -2.33616326e-02, -5.00978492e-02, -1.02615533e-02,
#         7.30055291e-03,  3.43292281e-02, -2.12623011e-02, -8.88561010e-02,
#         6.11365447e-03, -1.52316839e-02, -2.80353501e-02, -3.95468250e-02,
#        -3.22616100e-02, -1.33498395e-02,  3.01755173e-03, -1.85572114e-02,
#        -3.35369371e-02, -4.75610271e-02,  2.37506256e-02, -9.55078052e-04,
#        -2.07520872e-02,  6.79148063e-02, -1.58417374e-02, -4.85723931e-03,
#         5.24530327e-03, -4.89978446e-03,  9.10718646e-03,  1.87337515e-03,
#        -2.99504660e-02,  3.51611711e-03, -6.60958067e-02,  6.41788030e-03,
#         2.22590715e-02,  7.53074437e-02,  3.53788659e-02, -5.28311096e-02,
#         4.67569157e-02, -8.92656446e-02, -3.08805760e-02,  7.91029818e-03,
#         5.52251898e-02,  8.22758600e-02,  2.11164225e-02,  5.53627200e-02,
#         6.49390668e-02, -4.04138789e-02, -2.28957622e-04, -7.43037388e-02,
#         1.81916654e-02,  5.70268296e-02,  1.26477024e-02, -4.36689192e-03,
#         3.70855466e-03,  1.08027523e-02, -6.38372228e-02,  2.36931257e-02,
#        -3.11889723e-02, -4.69807237e-02,  1.30222142e-02,  4.47530393e-03,
#        -3.15080285e-02, -5.37434071e-02,  3.28278802e-02, -9.71143600e-03,
#        -3.07110866e-04,  8.38305950e-02, -1.95403229e-02,  5.09084342e-03,
#        -1.97637733e-02, -3.13787758e-02, -4.07412723e-02,  3.22756910e-04,
#        -8.61327630e-03, -3.75105515e-02, -4.59784232e-02,  6.17342107e-02,
#        -3.70969111e-03,  3.80500755e-03,  4.61920090e-02,  6.89788908e-02,
#        -6.14398606e-02,  4.04656213e-03,  1.93972848e-02, -8.83569419e-02,
#        -1.91511046e-02, -3.36884987e-03,  3.12168673e-02,  2.79408339e-02,
#         2.69571636e-02,  2.84631699e-02,  5.44577315e-02,  4.18368541e-03,
#        -4.36899811e-03, -5.60212275e-03,  1.00393472e-02,  2.35472689e-03,
#        -2.89809946e-02,  5.90746105e-03,  8.33062083e-03, -1.89391002e-02,
#        -3.17817368e-02,  1.79902986e-02,  1.18134692e-02,  6.12824596e-02,
#        -5.50248325e-02, -1.94575135e-02, -3.05044185e-03, -2.62853340e-03,
#        -4.01911959e-02, -3.97680327e-02,  5.51756844e-02,  3.65122147e-02,
#         9.05991625e-03,  3.22187170e-02,  3.10144504e-03, -2.33913977e-02,
#        -9.58415587e-03, -1.85423996e-03, -2.90373154e-02,  8.50840006e-03,
#         7.87928849e-02, -5.30835055e-02, -1.31328367e-02,  1.28030861e-02,
#         5.27247787e-02,  2.92754024e-02, -6.62970776e-03,  8.29944015e-02,
#         2.23353747e-02, -2.35364661e-02, -1.10355355e-02,  5.67874755e-04,
#         1.05503341e-02, -1.09110966e-01,  1.64827462e-02, -6.21259287e-02,
#         4.83694337e-02, -1.18169328e-02, -1.38406791e-02,  4.21534106e-02,
#         3.08368709e-02,  9.55001935e-02,  7.69733563e-02, -3.31638269e-02,
#         2.51529049e-02, -1.05492037e-03,  5.55963628e-02, -2.95078270e-02,
#        -8.78651626e-03, -3.61278467e-02,  3.98420058e-02,  2.47338936e-02,
#         4.00946736e-02,  2.51627900e-02, -1.98711511e-02,  5.13324104e-02,
#         1.96090732e-02, -4.06155810e-02, -4.59468178e-02,  6.02422394e-02,
#         1.91238876e-02,  1.41611304e-02,  1.49158379e-02,  8.63641035e-03,
#         1.24415727e-02, -2.85918228e-02, -3.90159003e-02, -2.25124732e-02,
#         3.73598821e-02,  3.57946986e-03, -1.60144060e-03, -9.97179300e-02,
#         1.41539723e-02, -5.37276566e-02,  9.28293914e-03, -5.70615567e-02,
#         1.60645917e-02, -1.79864559e-02, -3.83544788e-02, -2.85584480e-02,
#        -5.52239381e-02, -5.26455343e-02, -3.01631670e-02, -5.46472482e-02,
#        -1.45087689e-02, -8.18924140e-03,  3.82901356e-02, -3.51964347e-02,
#        -6.40672818e-02,  2.63199247e-02,  4.67148833e-02, -7.60204941e-02,
#         5.67530505e-02, -5.63785844e-02,  3.04889586e-02,  3.73169756e-03,
#         4.29984480e-02,  3.09336018e-02, -2.14926545e-02,  1.33711249e-02,
#         7.02268723e-03, -7.32180139e-04, -2.93228179e-02,  1.82911903e-02,
#         4.95046079e-02, -5.13419099e-02,  1.36850690e-02, -7.72360945e-03,
#         2.50036120e-02,  4.18160670e-02, -1.66418552e-02, -3.51010412e-02,
#         2.63861045e-02,  1.55906389e-02, -7.28965830e-03,  3.54819298e-02,
#        -4.35528830e-02,  4.94477078e-02,  2.14855727e-02, -3.27146202e-02,
#        -2.45707296e-03, -2.30731834e-02, -2.86313239e-04, -1.85405146e-02,
#        -3.26399994e-03, -6.73678704e-03, -2.20459364e-02, -1.77117512e-02,
#         2.99205836e-02, -5.36527149e-02, -2.90003549e-02, -2.83161178e-03,
#         1.82511397e-02, -4.49408330e-02,  5.60665242e-02,  1.03998818e-02,
#        -5.98358661e-02, -1.39806625e-02, -6.36844784e-02,  6.67121187e-02,
#        -1.58022512e-02,  5.68064116e-02,  5.82683599e-03,  6.09616600e-02,
#         6.19807839e-02,  2.15773266e-02, -6.15327805e-02,  1.66593213e-02,
#        -4.41254452e-02, -1.13536883e-02, -1.14786252e-02, -2.17435788e-02,
#        -1.97188631e-02, -4.91952747e-02,  9.38986689e-02,  5.29194474e-02,
#         2.54839361e-02, -5.69909476e-02, -7.05301389e-02,  3.56271155e-02,
#         2.58147027e-02, -5.36590107e-02,  2.65945662e-02,  7.41247982e-02,
#         3.92559692e-02, -6.16428368e-02,  2.26533599e-02, -3.93039472e-02,
#         4.47604023e-02,  3.72001491e-02, -5.27060404e-02, -5.08112200e-02,
#        -1.77795291e-02, -1.86528638e-02, -2.48605385e-02,  9.22461525e-02,
#        -3.99115905e-02,  3.14114057e-02, -2.24148408e-02, -6.05608104e-03,
#         4.59699798e-03,  6.55871257e-03,  1.82527874e-03, -8.05642735e-03,
#        -3.05913528e-03,  2.47964859e-02,  4.71892580e-02,  2.85140295e-02,
#         2.41699368e-02,  1.72727033e-02,  7.47751305e-03,  9.29769874e-02,
#        -7.47835189e-02,  1.83949247e-02, -8.50671157e-02, -5.06101709e-33,
#        -1.56907756e-02,  3.60210203e-02,  4.21261322e-03, -3.92971449e-02,
#        -3.22588049e-02,  2.03947648e-02,  6.77641183e-02,  8.95519275e-04,
#        -5.86214624e-02,  1.75790749e-02, -1.09964032e-02,  2.29445770e-02,
#         2.18059029e-02,  1.98075734e-02,  3.00855525e-02, -5.35030849e-03,
#         3.29179540e-02,  2.51705628e-02,  8.52273125e-03, -2.76516359e-02,
#         2.47744787e-02,  6.13632219e-05, -7.34299496e-02, -1.27970884e-02,
#         2.65562199e-02,  1.98717657e-02,  3.23472321e-02, -5.73546439e-02,
#        -1.58187002e-02, -1.52053703e-02,  4.43447530e-02,  2.79282425e-02,
#        -2.18765475e-02, -1.45105820e-03,  2.73276994e-04,  6.27390156e-03,
#        -2.46591195e-02, -4.01210180e-03,  2.26378143e-02,  3.14928032e-02,
#        -4.38302616e-03, -1.02146538e-02,  2.70649381e-02,  1.26182707e-02,
#         1.60928294e-02,  1.01417759e-02,  3.12726311e-02,  1.78395249e-02,
#        -5.88438287e-02,  3.23857996e-04,  3.15098427e-02, -1.09399436e-02,
#         1.30759673e-02,  2.88387947e-02, -1.98077876e-02,  4.71308567e-02,
#        -3.41147296e-02,  4.84242709e-03,  5.28466247e-04,  2.72621531e-02,
#         3.21750902e-03, -2.33742297e-02,  3.30080763e-02,  3.91170895e-03,
#        -1.13639552e-02,  9.49568115e-03,  2.39290129e-02, -2.84808762e-02,
#         4.16205525e-02, -2.69255531e-03, -4.31079008e-02,  3.59808095e-02,
#         5.03739156e-02, -2.85110250e-02, -2.84249466e-02,  4.52170242e-03,
#        -5.73280454e-02,  3.14072967e-02,  1.40482569e-02, -3.99632491e-02,
#        -9.78091061e-02,  7.01613203e-02, -2.46890783e-02,  2.98228581e-02,
#        -2.07746532e-02,  7.85104465e-04,  1.26001996e-03, -4.57690470e-02,
#        -9.89804231e-03,  1.88390445e-02, -1.46243749e-02,  3.76008116e-02,
#         2.75905207e-02, -1.09142335e-02,  9.39014554e-03, -5.32136969e-02,
#         5.25716972e-03,  3.80616374e-02, -3.88914496e-02, -2.09589079e-02,
#        -1.60054192e-02,  3.36383767e-02,  5.13529703e-02,  1.75285895e-04,
#         7.49087601e-04, -1.24496426e-02, -6.30771369e-02, -1.63804498e-02,
#        -3.62837948e-02, -1.08360564e-02, -3.04921344e-03,  5.65082878e-02,
#         6.99948892e-03,  2.80238315e-02,  5.78210726e-02,  4.96611977e-03,
#         3.77025874e-03, -2.84294374e-02, -3.26208137e-02, -6.75238594e-02,
#        -1.84433982e-02, -1.36077791e-01, -5.42167723e-02,  1.08281178e-02,
#        -1.77369249e-04,  3.75885814e-02, -1.12027479e-02,  3.95341814e-02,
#        -5.56724258e-02, -4.61753719e-02, -1.60338376e-02,  1.13554979e-02,
#         2.08576196e-07,  5.81366569e-02, -5.43791316e-02,  6.54743910e-02,
#         5.24706915e-02, -3.55250537e-02, -4.66826148e-02,  7.08852801e-03,
#         3.26832421e-02,  2.69151907e-02,  2.68478505e-02, -4.30063158e-02,
#        -7.80400354e-03, -3.43841431e-03,  5.41962758e-02,  2.54194494e-02,
#         7.08701741e-03,  1.75959466e-03,  8.17671418e-02, -2.86811162e-02,
#        -1.35912988e-02,  2.73381174e-02,  2.69771479e-02, -8.71380791e-03,
#         2.70841047e-02,  2.04075351e-02, -3.18201259e-03, -1.30775534e-02,
#        -1.41084762e-02,  5.35203237e-03, -7.44392499e-02, -1.27584515e-02,
#         1.18300496e-02, -5.95305264e-02,  1.56116411e-02, -6.03072438e-03,
#        -2.05468331e-02,  2.64899042e-02,  2.72203349e-02, -3.82143655e-04,
#         2.83557773e-02,  1.31781921e-02,  9.87495854e-03, -2.43306924e-02,
#        -2.53265142e-03,  6.56142682e-02,  3.04429140e-02,  1.30092725e-02,
#         1.81903429e-02, -1.87106710e-03, -3.11279558e-02,  9.91685688e-03,
#         1.95655841e-02, -1.80514937e-03,  2.22536065e-02,  2.46514054e-03,
#         9.54501238e-03, -5.87863429e-03,  2.27531651e-03, -5.49062863e-02,
#         1.68025296e-03,  2.45665144e-02,  1.04704984e-02, -3.63097899e-02,
#        -3.58516425e-02,  4.03851904e-02, -8.18642671e-04, -3.32758874e-02,
#         1.14417994e-34, -6.96108639e-02,  1.84117947e-02, -1.90845784e-02,
#         5.90707809e-02, -6.85224216e-03,  2.64697298e-02,  4.39649541e-03,
#        -3.31841619e-03, -3.24860751e-03,  7.86198583e-03, -6.80046761e-03],
#       "k": 5,
#       "num_candidates": 10000
#     }
#   }

In [24]:
response = es_client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"},
    },
    knn=knn_query,
    size=5
)

In [25]:
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'H49mGpEB8zadGYYapAui',
  '_score': 11.614713,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965448632836342,
    -0.0006260262452997267,
    -0.016629522666335106,
    0.05285142734646797,
    0.05476532131433487,
    -0.0313398540019989,
    0.029942654073238373,
    -0.04808565974235535,
    0.04467547684907913,
    0.0058394549414515495,
    0.016233116388320923,
    0.012001127004623413,
    -0.031222334131598473,
    0.01660059206187725,
    -0.04886896535754204,
    -0.06496302038431168,
    0.04643420875072479,
    -0.00929767545312643,
    -0.06425285339355469,
    -0.01373269222676754,
    -0.01597619242966175,
    0.008629539981484413,
    -0.024478962644934

#### References

1. https://logz.io/blog/elasticsearch-mapping/#:~:text=Within%20a%20search%20engine%2C%20mapping,indexes%20and%20stores%20its%20fields

2. https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

3. https://www.elastic.co/search-labs/tutorials

4. https://www.elastic.co/search-labs/blog/text-similarity-search-with-vectors-in-elasticsearch